# Forecast for Elme 

## Load data 

In [492]:
%%capture
!pip install awswrangler
!pip install mlforecast
!pip install lightgbm

In [493]:
import pandas as pd 
import numpy as np 
import os 
import boto3
import awswrangler as wr
from statsforecast import StatsForecast
import lightgbm as lgb

In [494]:
s3 = boto3.client('s3')
s3_path = 's3://amp-elme-sandbox/elme-filtered-us-ampfind-ovpc-data/time_series/adv_pub_ts_2023_07_24.csv' 
Y_df = wr.s3.read_csv(s3_path, boto3_session=boto3.Session())
Y_df.columns = Y_df.columns.str.lower()

In [495]:
Y_df = Y_df.rename(columns={'click_date': 'ds'})

## Data wrangling

In [496]:
unique_combs = [
    'adv_id',  
    'publisher_id',
]

In [497]:
# Create unique ids 
Y_df['unique_id'] = Y_df[unique_combs].astype(str).apply('_'.join, axis=1)

In [498]:
Y_df = Y_df.groupby(['unique_id', 'ds'])[['click_count', 'conv_count', 'order_value']].sum().reset_index()

In [499]:
Y_df['cvr'] = Y_df['conv_count'] / Y_df['click_count']
Y_df['aov'] = np.where(Y_df['conv_count'] == 0, 0, Y_df['order_value'] / Y_df['conv_count'])
Y_df['aov'] = Y_df['aov'].fillna(0)

In [500]:
Y_df.isna().mean()

unique_id      0.0
ds             0.0
click_count    0.0
conv_count     0.0
order_value    0.0
cvr            0.0
aov            0.0
dtype: float64

In [501]:
Y_df = pd.melt(
    Y_df, 
    id_vars=['unique_id', 'ds'], 
    value_vars=['cvr', 'aov'],
    var_name='kind',
    value_name='y',
)

In [502]:
Y_df['unique_id'] = Y_df[['unique_id', 'kind']].agg('_'.join, axis=1)

In [505]:
Y_df = Y_df.fillna(0)
Y_df = Y_df[Y_df.y != 0]

## Fill missing values

In [506]:
def fill_missing_values(df: pd.DataFrame, freq: str, max_date: str, min_date: str = None) -> pd.DataFrame:
    # Define a function to fill missing values in the time series data (in this case with 0s)
    # Get the minimum date in the DataFrame
    min_ds = df['ds'].iloc[0]

    # Create a new DataFrame with a complete date range and unique_id
    filled_df = pd.DataFrame({
        'unique_id': df['unique_id'].iloc[0],
        'ds': pd.date_range(min_ds if min_date is None else min_date, max_date, freq=freq)
    })

    # Convert date columns to strings for merging
    filled_df['ds'] = filled_df['ds'].astype(str)
    df['ds'] = df['ds'].astype(str)

    # Merge the original DataFrame with the filled DataFrame
    filled_df = filled_df.merge(df, how='left', on=['unique_id', 'ds'])

    # Fill missing 'y' values with 0
    filled_df['actual'] = filled_df['y']
    filled_df['y'] = filled_df['y'].ffill()

    # Convert the 'ds' column back to datetime
    filled_df['ds'] = pd.to_datetime(filled_df['ds'])

    return filled_df

In [507]:
Y_df['ds'].max()

'2023-07-23'

In [511]:
total_df = Y_df.groupby('unique_id').apply(fill_missing_values, "D", '2023-07-23').reset_index(drop=True)

### Load data here to skip EDA

In [512]:
total_df = total_df.sort_values(['unique_id', 'ds'])

## Modeling

In [513]:
total_df.kind = total_df['unique_id'].str[-3:]
total_df['kind'] = total_df['kind'].replace('vpc', 'ovpc')

In [520]:
# Remove all series that have had no sales in the last quarter (Q1 2023)
dates_lastQ = pd.date_range(start = '2023-05-01', end = '2023-06-30', freq = 'D').astype('str')

total_df['ds'] = total_df['ds'].astype('str')
train_df_wide = pd.pivot(total_df.query('kind == "cvr"'), index=['unique_id'], columns='ds', values='y')
train_df_wide['total'] = train_df_wide[dates_lastQ].sum(axis=1)
train_df_wide = train_df_wide[train_df_wide['total'] > 0]
train_df_wide = train_df_wide.reset_index()
total_df['ds'] = pd.to_datetime(total_df['ds'])

In [ ]:
ids_ts = train_df_wide['unique_id'].unique()
ids_ts = [id_.replace('_cvr', '') for id_ in ids_ts]

There are 322 time series that have had at least one sale in Q1 2023
This number represents 49.24% of the original number of time series


In [454]:
total_df['id'] = total_df['unique_id'].str.replace('_cvr', '').str.replace('_aov', '').str.replace('_ovpc', '')

In [456]:
total_sub_df = total_df.query('id in @ids_ts')

In [460]:
# remove series with less than 7 values
uids_to_remove = total_sub_df.groupby('unique_id').size().loc[lambda x: x <= 7 * 5].index
total_sub_df = total_sub_df.query('unique_id not in @uids_to_remove')

## MLForecast

In [463]:
from mlforecast import MLForecast
from lightgbm import LGBMRegressor
from sklearn.linear_model import Lasso, LinearRegression, Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from mlforecast.target_transforms import BaseTargetTransform, Differences
from window_ops.expanding import expanding_mean
from window_ops.rolling import rolling_mean
from numba import njit

@njit
def rolling_mean_7(x):
    return rolling_mean(x, window_size=7)

@njit
def rolling_mean_14(x):
    return rolling_mean(x, window_size=14)

@njit
def rolling_mean_28(x):
    return rolling_mean(x, window_size=28)

In [464]:
mlf = MLForecast(
    models={'LGBM': LGBMRegressor(objective='mape')},
    freq='D',
    lags=[1, 7, 14, 28],
    # lag_transforms={
    #     1: [expanding_mean],
    #     7: [rolling_mean_7, rolling_mean_14, rolling_mean_28],
    #     14: [rolling_mean_7, rolling_mean_14, rolling_mean_28],
    #     28: [rolling_mean_7, rolling_mean_14, rolling_mean_28]
    # },
    target_transforms=[Differences([1])],
    date_features=['quarter', 'month', 'week', 'dayofweek']
)

In [465]:
mlf_fcst = mlf.cross_validation(
    total_sub_df[['unique_id', 'ds', 'y']], 
    window_size=7,
    n_windows=4, 
    step_size=7, 
    refit=False
)

In [466]:
cv_df = mlf_fcst.merge(total_sub_df[["unique_id", 'ds', 'actual']])
cv_df = cv_df.dropna()
cv_df

,unique_id,ds,cutoff,y,LGBM,actual
14,73804_10915_aov,2023-06-26,2023-06-25,396.933333,229.500000,396.933333
16,73804_10915_aov,2023-06-28,2023-06-25,406.100000,229.500000,406.100000
18,73804_10915_aov,2023-06-30,2023-06-25,142.200000,229.500000,142.200000
19,73804_10915_aov,2023-07-01,2023-06-25,206.280000,229.500000,206.280000
21,73804_10915_cvr,2023-06-26,2023-06-25,0.054545,0.031957,0.054545
...,...,...,...,...,...,...
17928,8654_12911_cvr,2023-07-18,2023-07-16,0.071942,0.048800,0.071942
17929,8654_12911_cvr,2023-07-19,2023-07-16,0.059880,0.049344,0.059880
17930,8654_12911_cvr,2023-07-20,2023-07-16,0.059603,0.048788,0.059603
17931,8654_12911_cvr,2023-07-21,2023-07-16,0.062827,0.048894,0.062827


In [467]:
def mape(actuals, predictions):
    errors = np.abs((actuals - predictions) / actuals)
    mape = np.mean(errors) * 100
    return mape

cv_rmse = cv_df.groupby(['unique_id', 'cutoff']).apply(lambda df: mape(df['y'], df['LGBM'])).mean()
print("MAPE using cross-validation: ", cv_rmse)

MAPE using cross-validation:  73.29174331821473


In [479]:
train_df = total_sub_df[total_sub_df['ds'] <= pd.Timestamp('2023-06-29')]
test_df = total_sub_df[(total_sub_df['ds'] >= pd.Timestamp('2023-06-30')) & (total_sub_df['ds'] <= pd.Timestamp('2023-07-23'))]

In [480]:
mlf.fit(train_df[['unique_id', 'ds', 'y']])

MLForecast(models=[LGBM], freq=<Day>, lag_features=['lag1', 'lag7', 'lag14', 'lag28'], date_features=['quarter', 'month', 'week', 'dayofweek'], num_threads=1)

In [481]:
test_days[0]

Timestamp('2023-05-31 00:00:00')

In [482]:
total_prediction_days = pd.Series(test_df.ds.unique()).sort_values()
data = total_sub_df[['unique_id', 'ds', 'y', 'actual']]

def update_data (old_data, new_data):
    new_data = pd.concat([old_data, new_data], ignore_index=True)
    return new_data

def predict_current_date (model):
    predictions = model.predict(1)
    current_day_prediction = predictions.merge(data, on=['unique_id', 'ds'], how='left')
    current_day_prediction = current_day_prediction[['unique_id', 'ds', 'LGBM', 'actual']].rename(columns={'LGBM': 'y'})
    return current_day_prediction

def predict_next_date (model, previous_data):
    current_prediction = predict_current_date(model)
    new_data = update_data(previous_data, current_prediction)
    predictions = mlf.predict(horizon=1, new_data=new_data[['unique_id', 'ds', 'y']])
    predictions_df = predictions.merge(data, on=['unique_id', 'ds'], how='left')
    return predictions_df


previous_data = data[(data['ds'].dt.date <= total_prediction_days[0].date() - timedelta(days=1)) & (data['ds'].dt.date >= total_prediction_days[0].date() - timedelta(days=31))]
pred_df = predict_next_date(mlf, previous_data)

for day in total_prediction_days[:-1]:
    previous_data = data[(data['ds'].dt.date <= day.date() - timedelta(days=1)) & (data['ds'].dt.date >= day.date() - timedelta(days=31))]
    predictions_df = predict_next_date(mlf, previous_data)
    pred_df = pd.concat([pred_df, predictions_df], ignore_index=True)

In [483]:
forecast = pred_df[pred_df.y !=0].dropna().drop(columns = ["actual"])

In [484]:
forecast

,unique_id,ds,LGBM,y
2,73804_10915_aov,2023-07-01,406.100000,206.280000
3,73804_10915_cvr,2023-07-01,0.022589,0.016393
8,73804_11622_aov,2023-07-01,278.000000,292.300000
9,73804_11622_cvr,2023-07-01,0.014286,0.023529
30,73804_12755_aov,2023-07-01,576.342000,318.545000
...,...,...,...,...
15385,8654_12845_cvr,2023-07-22,0.055169,0.075163
15392,8654_12883_aov,2023-07-22,121.249677,93.682273
15393,8654_12883_cvr,2023-07-22,0.115877,0.108374
15400,8654_12911_aov,2023-07-22,175.184165,135.321667


In [485]:
mape(forecast['y'].values, forecast.LGBM.values)

64.36728800061505

In [486]:
forecast.to_parquet('forecast.parquet')